## Impressions and Reviews  
- unigram 1 word + tfidf 로 text feature화
- Logistic + CV 수행
- Logistic으로 multi-classification 수행시에는 predict_proba()[1]로 각 class의 확률을 구하여 결정

In [ ]:
import numpy as np
import pandas as pd

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_score
from scipy.sparse import hstack

class_names = ['toxic', 'severe_toxic', 'obscene', 'threat', 'insult', 'identity_hate']

train = pd.read_csv('../input/train.csv.zip').fillna(' ')
test = pd.read_csv('../input/test.csv.zip').fillna(' ')

train_text = train['comment_text']
test_text = test['comment_text']
all_text = pd.concat([train_text, test_text])

In [ ]:
word_vectorizer = TfidfVectorizer(
    sublinear_tf=True,
    strip_accents='unicode',
    analyzer='word',
    token_pattern=r'\w{1,}',
    stop_words='english',
    ngram_range=(1, 1),
    max_features=10000)
word_vectorizer.fit(all_text)
train_word_features = word_vectorizer.transform(train_text)
test_word_features = word_vectorizer.transform(test_text)

In [ ]:
train_word_features.shape, test_word_features.shape

In [ ]:
# 1 unigram (1 word)
print(sorted(word_vectorizer.vocabulary_.items(),reverse =True)[:20])
print(sorted(word_vectorizer.vocabulary_.items(),reverse =False)[1000])
print(sorted(word_vectorizer.vocabulary_.items(),reverse =False)[:20])

In [ ]:
char_vectorizer = TfidfVectorizer(
    sublinear_tf=True,
    strip_accents='unicode',
    analyzer='char',
    stop_words='english',
    ngram_range=(2, 6),
    max_features=50000)

char_vectorizer.fit(all_text)
train_char_features = char_vectorizer.transform(train_text)
test_char_features = char_vectorizer.transform(test_text)

In [ ]:
# 1 unigram (1 word)
print(sorted(word_vectorizer.vocabulary_.items(),reverse = True)[:20])
print(sorted(word_vectorizer.vocabulary_.items(),reverse = False)[1000])
print(sorted(word_vectorizer.vocabulary_.items(),reverse = False)[:20])

In [ ]:
train_features = hstack([train_char_features, train_word_features])
test_features = hstack([test_char_features, test_word_features])
scores = []
submission = pd.DataFrame.from_dict({'id': test['id']})

In [ ]:
for class_name in class_names:
    train_target = train[class_name]
    classifier = LogisticRegression(C=0.1, solver='sag')

    cv_score = np.mean(cross_val_score(classifier, train_features, train_target, cv=3, scoring='roc_auc'))
    scores.append(cv_score)
    print('CV score for class {} is {}'.format(class_name, cv_score))

    classifier.fit(train_features, train_target)
    submission[class_name] = classifier.predict_proba(test_features)[:, 1]

print('Total CV score is {}'.format(np.mean(scores)))

In [ ]:
submission.to_csv('submission.csv', index=False)